<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/scripts/elinor_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose

This file shows the steps we took to sample and create the annotation dataset.

## Connect with Google drive to access data 

In order to access the data, you first need to create a shortcut of the data folder to your own Gdrive. If you've been granted editing rights, you should be able to edit the content of the folder, i.e. add, move and delete data, create and rename folders, etc.

In [1]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# redirect the working directory of this script to the data folder
%cd /content/drive/MyDrive/data/

/content/drive/MyDrive/data


## Load data

### Method 1: get csv files 

In [56]:
import tqdm as tqdm
import os
import pandas as pd
import re 

folder_path = "filtered2"

In [ ]:
dfs = []

# loop through files 
for filename in os.listdir(folder_path):
    # if csv file, load and add to dfs  
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

# combine files in df
df = pd.concat(dfs, ignore_index=True)

### Method 2: get a csv file

In [96]:
import tqdm as tqdm
import os
import pandas as pd

df_subset = pd.read_csv('sample.csv', encoding='utf-8')

## Inspect data


In [98]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  50 non-null     int64 
 1   artikel_id  50 non-null     object
 2   name        50 non-null     object
 3   jahrgang    50 non-null     int64 
 4   datum       50 non-null     int64 
 5   ressort     48 non-null     object
 6   titel       50 non-null     object
 7   untertitel  22 non-null     object
 8   text        50 non-null     object
dtypes: int64(3), object(6)
memory usage: 3.6+ KB


In [99]:
# remove first column 
df_subset1 = df_subset.drop("Unnamed: 0", axis=1)

## Create a random subset of the data 

In [ ]:
# size of subset we want 
number = 100

In [ ]:
perc = number/df1.shape[0]
df_subset = df1.sample(frac=perc, random_state=42)

In [ ]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 141433 to 1157830
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   artikel_id  100 non-null    object 
 1   name        100 non-null    object 
 2   jahrgang    100 non-null    object 
 3   datum       100 non-null    float64
 4   ressort     89 non-null     object 
 5   titel       92 non-null     object 
 6   untertitel  37 non-null     object 
 7   text        100 non-null    object 
dtypes: float64(1), object(7)
memory usage: 7.0+ KB


## Adjust format for export

### String fixes 

In [100]:
# remove square brackets 
df_subset1["text"] = df_subset1["text"].apply(lambda x: x[1:-1] if (isinstance(x, str) and x.startswith("[") and x.endswith("]")) else x)
# remove backward slashes
df_subset1['text'] = df_subset1['text'].str.replace('\\', '', regex = False)
# remove genios styles
df_subset1["text"] = df_subset1["text"].apply(lambda x: re.sub(r'<.*?>', '', x))

In [101]:
# function to split strings to list
def split_to_list(s):
    return s.split("', '")

In [102]:
# apply function to whole column
df_subset1['text'] = df_subset1['text'].apply(split_to_list)

In [103]:
# add new line to separate paragraphs 
df_subset1['text'] = df_subset1['text'].apply(lambda x: [char+'\n' for char in x])

### Split by paragraphs

In [ ]:
# Explode "text" column
df_elinor = df_subset1.explode("text")

# Filter out paragraphs with less than 5 words
df_elinor1 = df_elinor[df_elinor["text"].apply(lambda x: len(x.split()) >= 5)]

# Create "artikel_order" column
df_elinor1["artikel_order"] = df_elinor1.groupby("artikel_id").cumcount() + 1


### Randomly select one paragraph per article

In [105]:
# function to randomly select one row
def random_select(group):
    return group.sample(n=1, random_state=1)


In [106]:
# Apply function to randomly select one row per unique artikel_id
df_elinor2 = df_elinor1.groupby("artikel_id").apply(random_select).reset_index(drop=True)


In [107]:
len(df_elinor2)

50

In [108]:
df_elinor3 = df_elinor2[['text']]

## Export as csv

In [109]:
output_path = "elinor"

In [111]:
df_elinor3.to_csv(output_path+"/annotation_test3.csv", index=False, header = True,
                  encoding = 'utf-8',
                  sep="\t")